In [1]:
import numpy as np
import glob
import os

from pathlib import Path
import cv2
import numpy as np
from matplotlib.animation import ArtistAnimation
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from sklearn.manifold import TSNE
from tqdm import tqdm
from enum import Enum
from typing import Optional
from r3m import load_r3m

import torch
import torch.nn as nn
import torch.multiprocessing as mp
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize, ToPILImage

from dataset import CustomDataset, AttrDict

# datapath_dd_training = '/media/tim/6f37312f-8eb4-400c-a4e7-e229c18bbf2c/datasets/calvin_debug_dataset/training'
# datapath_dd_training = '/media/tim/E/datasets/task_D_D/training'
# datapath_dd_val = '/media/tim/E/datasets/task_D_D/validation'
datapath_dd_training = '/home/tim/calvin_debug_dataset/training'
datapath_dd_val = '/home/tim/calvin_debug_dataset/validation'
# datapath_dd_training = '/media/tim/6f37312f-8eb4-400c-a4e7-e229c18bbf2c/datasets/hulc2/unprocessed/real_world/500k_all_tasks_dataset_15hz'
mp.set_start_method('spawn')


/home/tim/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-26 00:30:09.743679: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 00:30:09.889526: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 00:30:09.891285: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-26 00:30:10.923419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Visualize Calvin Data

In [4]:
def visualize():

    #parser = ArgumentParser(description="Interactive visualization of CALVIN dataset")
    #parser.add_argument("path", type=str, help="Path to dir containing scene_info.npy")
    #parser.add_argument("-d", "--data", nargs="*", default=["rgb_static", "rgb_gripper"], help="Data to visualize")
    #args = parser.parse_args()

    path = datapath_dd_training
    data = ["rgb_static", "rgb_gripper"]

    if not Path(path).is_dir():
        print(f"Path {path} is either not a directory, or does not exist.")
        exit()

    indices = next(iter(np.load(f"{path}/scene_info.npy", allow_pickle=True).item().values()))
    indices = list(range(indices[0], indices[1] + 1))

    scene_info = np.load(f"{path}/scene_info.npy", allow_pickle=True)
    print(scene_info)

    annotations = np.load(f"{path}/lang_annotations/auto_lang_ann.npy", allow_pickle=True).item()
    annotations = list(zip(annotations["info"]["indx"], annotations["language"]["ann"]))
    print(annotations)
    print(len(annotations))

    idx = 0
    # idx = 60000
    ann_idx = -1

    while True:
        t = np.load(f"{path}/episode_{indices[idx]:07d}.npz", allow_pickle=True)

        for d in data:
            if d not in t:
                print(f"Data {d} cannot be found in transition")
                continue

            img = cv2.resize(t[d], (400, 400))
            cv2.imshow(d, img[:, :, ::-1])

        for n, ((low, high), ann) in enumerate(annotations):
            if indices[idx] >= low and indices[idx] <= high:
                if n != ann_idx:
                    print(f"{ann}")
                    ann_idx = n

        # user_input = input("Enter something: ")


        key = cv2.waitKey(0)
        if key == ord("q"):
            break
        elif key == 83:  # Right arrow
            idx = (idx + 1) % len(indices)
        elif key == 81:  # Left arrow
            idx = (len(indices) + idx - 1) % len(indices)
        else:
            print(f'Unrecognized keycode "{key}"')

        

In [5]:
visualize()

{'calvin_scene_D': [358482, 361252]}
[((358656, 358720), 'move the light switch to turn on the yellow light'), ((359714, 359757), 'sweep the pink block to the right'), ((360978, 361011), 'place the block in the sliding cabinet'), ((358728, 358792), 'pick up the red block from the table'), ((359534, 359598), 'in the slider grasp the blue block'), ((360571, 360635), 'slide down the switch'), ((358729, 358793), 'pick up the red block on the table'), ((359069, 359103), 'place in slider'), ((360575, 360639), 'turn off the light bulb')]
9


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/tim/.local/lib/python3.10/site-packages/cv2/qt/plugins"


In [2]:
datapath_test= '/media/tim/E/datasets/task_D_D/training/episode_0053819.npz'
data = np.load(datapath_test)
print(list(data.keys()))
print(data['actions'].shape)
print(data['rgb_static'].shape)
print(data['rgb_gripper'].shape)

FileNotFoundError: [Errno 2] No such file or directory: '/media/tim/E/datasets/task_D_D/training/episode_0053819.npz'

## Captioning
### Data Loader

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
max_seq_length = 16

path = '/home/tim/calvin_debug_dataset/'

datapath_dd_training = '{}training'.format(path)
datapath_dd_val = '{}validation'.format(path)
caption_path_training = '{}training/lang_annotations/auto_lang_ann.npy'.format(path)
caption_path_val = '{}validation/lang_annotations/auto_lang_ann.npy'.format(path)

train_dataset = CustomDataset(datapath_dd_training, caption_path_training, tokenizer, max_seq_length)
val_dataset  = CustomDataset(datapath_dd_val, caption_path_val, tokenizer, max_seq_length)

batch_size = 32
num_workers = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)


/home/tim/.local/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
/home/tim/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tim/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Test Data Loader

In [3]:

"""
for batch in train_dataloader:
   # print("token: ", batch.token.shape)
    print("gpt_tokens: ", batch.gpt_tokens.shape)
    print("gpt_mask: ", batch.gpt_mask.shape)
    print("gpt_tokens: ", batch.gpt_tokens[0])
    print("gpt_mask: ", batch.gpt_mask[0])
   # if(torch.all(torch.eq(batch.gpt_tokens, batch.token))):
   #     print("correct")
    print(batch.instruction[0])
    print("actions: ", batch.actions.shape)
    print("rgb_static: ", batch.rgb_static.shape)
    print("rgb_gripper: ", batch.rgb_gripper.shape)
    print("#############################################")
    break
"""

for batch in train_dataloader:
   # print("token: ", batch.token.shape)
    print("gpt_tokens: ", batch.gpt_tokens.shape)
    print("gpt_mask: ", batch.gpt_mask.shape)
    print("gpt_tokens: ", batch.gpt_tokens[0])
    print("gpt_mask: ", batch.gpt_mask[0])
   # if(torch.all(torch.eq(batch.gpt_tokens, batch.token))):
   #     print("correct")
    print(batch.instruction[0])
    print("actions: ", batch.actions.shape)
    print("observations: ", batch.observations.shape)
    print("observations: ", batch.observations)
    print("#############################################")
    break

2023-08-26 00:30:23.990830: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 00:30:24.033202: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 00:30:24.033566: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-26 00:30:25.004452: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-26 00:30:28.084354: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 00:30:28.126621: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-26 00:30:28.

gpt_tokens:  torch.Size([4, 16])
gpt_mask:  torch.Size([4, 16])
gpt_tokens:  tensor([15344,   572,   262,  1657, 28287,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0])
gpt_mask:  tensor([1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
turn off the light bulb
actions:  torch.Size([4, 64, 7])
observations:  torch.Size([4, 6, 2048])
observations:  tensor([[[1.0301e-03, 0.0000e+00, 1.1530e-02,  ..., 7.2582e-03,
          5.0681e-02, 8.3216e-02],
         [6.6977e-03, 0.0000e+00, 5.4827e-03,  ..., 1.1312e-02,
          1.5491e-02, 8.1042e-02],
         [4.1995e-03, 0.0000e+00, 3.8414e-03,  ..., 1.0868e-02,
          3.4670e-02, 9.4621e-02],
         [1.0301e-03, 0.0000e+00, 1.1530e-02,  ..., 7.2582e-03,
          5.0681e-02, 8.3216e-02],
         [6.6977e-03, 0.0000e+00, 5.4827e-03,  ..., 1.1312e-02,
          1.5491e-02, 8.1042e-02],
         [4.1995e-03, 0.0000e+00, 3.8414e-03,  ..., 1.0868e-02,
          3.4670e-02, 9.4621e-02]],

  

### Models
Captioning Model

In [ ]:

class MappingType(Enum):
    MLP = 'mlp'
    Transformer = 'transformer'

class BehaviourEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        # self.pos_encoder = PositionalEncoding(CFG.d_model, dropout=CFG.dropout)
        self.pos_encoder = PositionalEncoding(2050, dropout=0.1)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=CFG.d_model, nhead=CFG.n_heads, batch_first=True, dim_feedforward=CFG.d_ff, dropout=CFG.dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=CFG.n_layers)

    def forward(self, src):

        image_features = src.observations # images: [batch_size, sequence_length, 2048]
        actions = src.actions

        src_key_padding_mask = (image_features.mean(dim=2)==0.0)
        features = torch.cat((image_features, actions), dim=-1)

        # Transformer encoder
        # add positional encoding
        features = features * math.sqrt(CFG.d_model)mp.set_start_method('spawn')
        features = self.pos_encoder(features)
        behaviour_encoding = self.transformer_encoder(features, src_key_padding_mask=src_key_padding_mask)

        return behaviour_encoding


class ClipCaptionModel(nn.Module):

    def __init__(self, prefix_length: int, clip_length: Optional[int] = None, prefix_size: int = 512,
                 num_layers: int = 8, mapping_type: MappingType = MappingType.MLP):
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = prefix_length
        self.gpt = GPT2LMHeadModel.from_pretrained('gpt2')
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        self.clip_project = TransformerMapper(prefix_size, self.gpt_embedding_size, prefix_length,
                                                                     clip_length, num_layers)
        self.behaviour_encoder = BehaviourEncoder()
        self.project_to_gpt = nn.Linear(514, self.gpt_embedding_size).to("cuda")

    def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)

    def forward(self, data: AttrDict):

        tokens = data.gpt_tokens
        gpt_mask = data.gpt_mask
        labels = None

        embedding_text = self.gpt.transformer.wte(tokens)
        behaviour_encoding  = self.behaviour_encoder(data)
        
        # !!!!!!!!!!!
        behaviour_encoder_padding_mask = ~(data.rgb_static.mean(dim=2)==0.0) * 1.0

        prefix_projections = self.project_to_gpt(behaviour_encoding)

        total_mask = torch.cat((behaviour_encoder_padding_mask, embedding_text), dim=1)

        ######################################
        # prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=total_mask)
        return out